# WEEK 1 CAPSTONE PROGRESSS
This is the first phase of the model development progress, in this phase we will try to design the base architecture of the model for our 'UrKins" application

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# Memuat model MobileNetV2 dengan bobot ImageNet tanpa lapisan atas (top layer)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Baca file CSV yang berisi label
df = pd.read_csv('path_to_dataset/labels.csv')

# Membagi data menjadi training, validation, dan testing
train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)  # 0.2 dari train_df untuk validasi

# Membuat generator data gambar
datagen = ImageDataGenerator(rescale=1./255)

# Fungsi untuk menghasilkan data training dan validation
def create_generator(datagen, dataframe, subset):
    return datagen.flow_from_dataframe(
        dataframe=dataframe,
        directory='path_to_dataset/images',
        x_col='filename',
        y_col=['kulit_kering', 'kulit_berminyak', 'kulit_keriput', 'kulit_berjerawat', 'kulit_merah'],
        target_size=(224, 224),
        batch_size=32,
        class_mode='raw',
        subset=subset
    )

# Generator data untuk training, validation, dan testing
train_generator = create_generator(datagen, train_df, None)
validation_generator = create_generator(datagen, val_df, None)
test_generator = create_generator(datagen, test_df, None)

# Model pelatihan dan evaluasi tetap sama
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10
)

# Evaluasi model pada data testing
loss, accuracy = model.evaluate(test_generator)
print(f'Testing Loss: {loss}, Testing Accuracy: {accuracy}')


In [ ]:
# Menambahkan lapisan kustom di atas MobileNetV2
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
outputs = Dense(5, activation='sigmoid')(x)  # Menggunakan sigmoid untuk multi-label classification

# Membuat model akhir
model = Model(inputs=base_model.input, outputs=outputs)

In [ ]:
# Membekukan semua lapisan dari model MobileNetV2
for layer in base_model.layers:
    layer.trainable = False

# Mengompilasi model
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)  # Memisahkan data untuk validasi

# Menyiapkan generator data
train_generator = train_datagen.flow_from_directory(
    'path_to_dataset',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training')  # Data pelatihan

validation_generator = train_datagen.flow_from_directory(
    'path_to_dataset',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation')  # Data validasi

validation_generator = train_datagen.flow_from_directory(
    'path_to_dataset',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='test')  # Data Testing

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'path_to_dataset'

In [ ]:
# Melatih model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10)


In [ ]:
# Fine-tuning: Membuka beberapa lapisan terakhir dari MobileNetV2
for layer in base_model.layers[-10:]:
    layer.trainable = True

# Mengompilasi ulang model
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Melanjutkan pelatihan model dengan fine-tuning
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10)

In [ ]:

# Evaluasi model
loss, accuracy = model.evaluate(validation_generator)
print(f'Loss: {loss}, Accuracy: {accuracy}')